# Explore pybaseball

In [2]:
from pybaseball import playerid_lookup, statcast_pitcher, statcast_pitcher_spin

In [9]:
player = playerid_lookup('bauer', 'trevor')['key_mlbam'].values[0]

In [10]:
player

545333

In [11]:
player_stats = statcast_pitcher('2021-01-01', '2021-06-03', player)

Gathering Player Data


In [13]:
player_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 92 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   pitch_type                       1255 non-null   object 
 1   game_date                        1255 non-null   object 
 2   release_speed                    1255 non-null   float64
 3   release_pos_x                    1255 non-null   float64
 4   release_pos_z                    1255 non-null   float64
 5   player_name                      1255 non-null   object 
 6   batter                           1255 non-null   int64  
 7   pitcher                          1255 non-null   int64  
 8   events                           294 non-null    object 
 9   description                      1255 non-null   object 
 10  spin_dir                         0 non-null      float64
 11  spin_rate_deprecated             0 non-null      float64
 12  break_angle_deprecat

In [16]:
player_stats[['player_name', 'pitch_type', 'release_speed', 'release_spin_rate']]

,player_name,pitch_type,release_speed,release_spin_rate
0,"Bauer, Trevor",FC,82.3,2906.0
1,"Bauer, Trevor",FC,84.3,2994.0
2,"Bauer, Trevor",FC,83.5,3061.0
3,"Bauer, Trevor",FF,92.2,2882.0
4,"Bauer, Trevor",SL,78.8,2902.0
...,...,...,...,...
1250,"Bauer, Trevor",FF,94.1,2700.0
1251,"Bauer, Trevor",FF,92.3,2694.0
1252,"Bauer, Trevor",FF,91.2,2671.0
1253,"Bauer, Trevor",FF,92.2,2736.0


In [17]:
# add SVR (=spinrate/speed)
player_stats['SVR'] = player_stats['release_spin_rate'] / player_stats['release_speed']
player_stats[['player_name', 'pitch_type', 'game_date', 'SVR']]

,player_name,pitch_type,game_date,SVR
0,"Bauer, Trevor",FC,2021-05-31,35.309842
1,"Bauer, Trevor",FC,2021-05-31,35.516014
2,"Bauer, Trevor",FC,2021-05-31,36.658683
3,"Bauer, Trevor",FF,2021-05-31,31.258134
4,"Bauer, Trevor",SL,2021-05-31,36.827411
...,...,...,...,...
1250,"Bauer, Trevor",FF,2021-04-02,28.692880
1251,"Bauer, Trevor",FF,2021-04-02,29.187432
1252,"Bauer, Trevor",FF,2021-04-02,29.287281
1253,"Bauer, Trevor",FF,2021-04-02,29.674620


In [18]:
# selct for pitches: FT, FF, CU, FC
X = player_stats[player_stats['pitch_type'].isin(['FT', 'FC', 'CU', 'FC'])]

In [21]:
X.shape

(260, 93)